In [1]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[{'label':'All Sites','value':'ALL'},
                                {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}],
                                value ='ALL',
                                placeholder='Select Launch Site here :',
                                searchable=True),
                                 
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))

def render_pie(site_dropdown):

    if site_dropdown == 'ALL':      
        fig = px.pie(spacex_df, values='class', names='Launch Site', title="Success launches for site")
        return fig

    else:
        dropdown_pie= spacex_df[spacex_df['Launch Site'] == site_dropdown]
        dropdown_pie_class = dropdown_pie.groupby(['Launch Site','class']).size().reset_index(name='class count')       
        fig = px.pie(dropdown_pie_class, values='class count', names='class', title="Success launches for site")
        return fig

@app.callback( Output(component_id='success-payload-scatter-chart',component_property='figure'),
 [Input(component_id='site-dropdown',component_property='value'),
  Input(component_id='payload-slider',component_property='value')]) 

def render_scatter(site_dropdown,slider_range):
    
    low, high = slider_range
    slide = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    dropdown_scatter=spacex_df[slide]

    if site_dropdown == 'ALL':
        title_scatter = f"Payload and Booster Versions for site: {site_dropdown}"
        fig = px.scatter(dropdown_scatter)
        return fig

    else:
        dropdown_scatter1= dropdown_scatter[dropdown_scatter['Launch Site'] == site_dropdown]
        title_scatter = f"Payload and Booster Versions for site {site_dropdown}"
        fig = px.scatter(dropdown_scatter1, x="Payload Mass (kg)", y="class", color="Booster Version Category", title=title_scatter)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

<ipython-input-2-75809e5439eb>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-2-75809e5439eb>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
